# DATA PREPROCESSING
The purpose of this part is to transform variables into the usable format.

In [1]:
import pandas as pd
import numpy as np
import datetime
import pytz
import us
from timezonefinder import TimezoneFinder
from geopy.geocoders import Nominatim

In [2]:
pd.options.display.max_columns = 200
pd.set_option('mode.chained_assignment', None)

In [3]:
data = pd.read_csv('flights.csv')
airports = pd.read_csv('airports.csv')

C:\Users\ja\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


We will use only the flights from the first four months (from January to April). We don't have to care about the warning. ORIGIN_AIRPORT and DESTINATION_AIRPORT are in the format of three letters but in October they are stored as digits. They are in the format of FAA scale of airports instead of IATA as in the other months.

We could correct October based on the data from https://www.transtats.bts.gov/DL_SelectFields.asp. The column OriginAirportID is in the FAA scale and Origin indicates the IATA scale so we could easily replace October data. However, we will only use the data from the first four months so we don't have to correct it.

In [4]:
data = data[data.MONTH <= 4]

In [5]:
data.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


Let's merge flights with airports. 

In [7]:
airports_to_merge = ['IATA_CODE', 'STATE', 'LATITUDE', 'LONGITUDE', 'CITY']

data = pd.merge(data, airports[airports_to_merge], how = 'left', left_on = 'ORIGIN_AIRPORT', right_on = 'IATA_CODE')
data.rename(columns = {'STATE': 'STATE_ORIGIN', 'LATITUDE': 'LATITUDE_ORIGIN', 'LONGITUDE': 'LONGITUDE_ORIGIN', 'CITY': 'CITY_ORIGIN'}, inplace = True)
data.drop('IATA_CODE', axis = 1, inplace = True)

data = pd.merge(data, airports[airports_to_merge], how = 'left', left_on = 'DESTINATION_AIRPORT', right_on = 'IATA_CODE')
data.rename(columns = {'STATE': 'STATE_DESTINATION', 'LATITUDE': 'LATITUDE_DESTINATION', 'LONGITUDE': 'LONGITUDE_DESTINATION', 'CITY': 'CITY_DESTINATION'}, inplace = True)
data.drop('IATA_CODE', axis = 1, inplace = True)

# MISSING ARRIVAL DELAY

In [8]:
print('Count of flights with missing arrival delay:', len(data[data.ARRIVAL_DELAY.isna()]))
print('Count of diverted or caneled flights:', len(data[(data.DIVERTED == 1) | (data.CANCELLED == 1)]))

Count of flights with missing arrival delay: 52557
Count of diverted or caneled flights: 52557


We are predicting arrival delay for flights so we should remove values that don't contain information on arrival delay. As we can see such flights are the flights that are diverted or cancelled.

Let's drop those values and columns concerning diverted and cancelled flights.

In [9]:
data = data[data.ARRIVAL_DELAY.isna() == False]
data.reset_index(drop = True, inplace = True)

data = data.drop(['DIVERTED', 'CANCELLED', 'CANCELLATION_REASON'], axis = 1)

# VARIABLES CONCERNING TIME

In [10]:
time_based_columns = ['SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'SCHEDULED_ARRIVAL', 'WHEELS_OFF', 'ARRIVAL_TIME']
data[time_based_columns].head()

,SCHEDULED_DEPARTURE,DEPARTURE_TIME,SCHEDULED_ARRIVAL,WHEELS_OFF,ARRIVAL_TIME
0,5,2354.0,430,15.0,408.0
1,10,2.0,750,14.0,741.0
2,20,18.0,806,34.0,811.0
3,20,15.0,805,30.0,756.0
4,25,24.0,320,35.0,259.0


As we can see, variables concerning time are in the format of HHMM, not separated in any way.
We will encode those variables into separated columns including respectively hours and minutes.

In [11]:
def time_to_HH_MM_cols(data, col):
    data.loc[(data[col] >= 0) & (data[col] < 60), (col + '_HH')] = 0
    data.loc[(data[col] >= 0) & (data[col] < 60), (col + '_MM')] = data[col]
    data.loc[(data[col] >= 100) & (data[col] < 960), (col +'_HH')] = data[col].astype('str').str[0]
    data.loc[(data[col] >= 100) & (data[col] < 960), (col + '_MM')] = data[col].astype('str').str[1:3]
    data.loc[(data[col] >= 1000), (col + '_HH')] = data[col].astype('str').str[0:2]
    data.loc[(data[col] >= 1000), (col + '_MM')] = data[col].astype('str').str[2:4]

In [12]:
for col in time_based_columns:
    time_to_HH_MM_cols(data, col)

Let's also change newly created variables into integers.

In [13]:
HH_MM_cols = data.columns[data.columns.str.endswith('HH') | data.columns.str.endswith('MM')]

for col in HH_MM_cols:    
    data[col] = data[col].astype('int8')

# TIME ZONES

Our objective is to create normalized dates of departure and arrival in order to obtain information on recent flights, on which we can then perform aggregations and statistics.

We are able to derive right know arrival dates in the departure time zone as a sum of a departure time and an elapsed time of the flight. We don't base on arrival time due to the cases when the day of departure time doesn't match the day of arrival time.

In [14]:
data['SCHEDULED_DEPARTURE'] = pd.to_datetime(dict(year = data.YEAR, month = data.MONTH, day = data.DAY,
                                                  hour = data.SCHEDULED_DEPARTURE_HH, minute = data.SCHEDULED_DEPARTURE_MM))
data['DEPARTURE_TIME'] = data['SCHEDULED_DEPARTURE'] + pd.to_timedelta(data.DEPARTURE_DELAY, unit = 'minutes')
data['ARRIVAL_TIME_ORIGIN_TZ'] = data.DEPARTURE_TIME + pd.to_timedelta(data.ELAPSED_TIME, unit = 'minutes')

In [15]:
data.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,STATE_ORIGIN,LATITUDE_ORIGIN,LONGITUDE_ORIGIN,CITY_ORIGIN,STATE_DESTINATION,LATITUDE_DESTINATION,LONGITUDE_DESTINATION,CITY_DESTINATION,SCHEDULED_DEPARTURE_HH,SCHEDULED_DEPARTURE_MM,DEPARTURE_TIME_HH,DEPARTURE_TIME_MM,SCHEDULED_ARRIVAL_HH,SCHEDULED_ARRIVAL_MM,WHEELS_OFF_HH,WHEELS_OFF_MM,ARRIVAL_TIME_HH,ARRIVAL_TIME_MM,ARRIVAL_TIME_ORIGIN_TZ
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,2015-01-01 00:05:00,2014-12-31 23:54:00,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,NaN,NaN,NaN,NaN,NaN,AK,61.17432,-149.99619,Anchorage,WA,47.44898,-122.30931,Seattle,0,5,23,54,4,30,0,15,4,8,2015-01-01 03:08:00
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,2015-01-01 00:10:00,2015-01-01 00:02:00,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,NaN,NaN,NaN,NaN,NaN,CA,33.94254,-118.40807,Los Angeles,FL,26.68316,-80.09559,West Palm Beach,0,10,0,2,7,50,0,14,7,41,2015-01-01 04:41:00
2,2015,1,1,4,US,840,N171US,SFO,CLT,2015-01-01 00:20:00,2015-01-01 00:18:00,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,NaN,NaN,NaN,NaN,NaN,CA,37.61900,-122.37484,San Francisco,NC,35.21401,-80.94313,Charlotte,0,20,0,18,8,6,0,34,8,11,2015-01-01 05:11:00
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,2015-01-01 00:20:00,2015-01-01 00:15:00,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,NaN,NaN,NaN,NaN,NaN,CA,33.94254,-118.40807,Los Angeles,FL,25.79325,-80.29056,Miami,0,20,0,15,8,5,0,30,7,56,2015-01-01 04:56:00
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,2015-01-01 00:25:00,2015-01-01 00:24:00,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,NaN,NaN,NaN,NaN,NaN,WA,47.44898,-122.30931,Seattle,AK,61.17432,-149.99619,Anchorage,0,25,0,24,3,20,0,35,2,59,2015-01-01 03:59:00


There are states in the United States where there exist a few different time zones. Thus, states aren't the best indicator of time zones.

Instead of that, we can use coordinates of cities. There is a package timezonefinder that handle transformation of longitudes and latitudes into time zones.

Let's check first whether there are any missing values for latitudes and longitudes.

In [16]:
print('Missing values of latitude of origin:',data.LATITUDE_ORIGIN.isna().sum())
print('Missing values of longitude origin:',data.LONGITUDE_ORIGIN.isna().sum())

print('Missing values of latitude of destination:',data.LATITUDE_DESTINATION.isna().sum())
print('Missing values of longitude of destination:',data.LONGITUDE_DESTINATION.isna().sum())

print('Missing values of city of origin:', data.CITY_ORIGIN.isna().sum())
print('Missing values of city of destination:', data.CITY_DESTINATION.isna().sum())

Missing values of latitude of origin: 1442
Missing values of longitude origin: 1442
Missing values of latitude of destination: 1446
Missing values of longitude of destination: 1446
Missing values of city of origin: 0
Missing values of city of destination: 0


There are missing values both for latitudes and longitudes. We can derive those values from cities using geopy package.

There aren't any nans for cities.

In [17]:
cities_origin_without_coords = data.loc[data.LATITUDE_ORIGIN.isna() | data.LONGITUDE_ORIGIN.isna(),
                                        'CITY_ORIGIN'].unique()
cities_destination_without_coords = data.loc[data.LATITUDE_DESTINATION.isna() | data.LONGITUDE_DESTINATION.isna(),
                                             'CITY_DESTINATION'].unique()

print('Origin cities without coordinates:', cities_origin_without_coords)
print('Destination cities without coordinates:', cities_destination_without_coords)

Origin cities without coordinates: ['Plattsburgh' 'Panama City' 'St. Augustine']
Destination cities without coordinates: ['Panama City' 'St. Augustine' 'Plattsburgh']


As we can see there are 3 cities that lack coordinates. Let's get coordinates for them.

In [18]:
geolocator = Nominatim(user_agent="my-application")
def geolocate(city):
    loc = geolocator.geocode(str(city + ',' + 'United States'))
    return loc.latitude, loc.longitude

In [19]:
for i in cities_origin_without_coords:
    data.loc[data.CITY_ORIGIN == i, ['LATITUDE_ORIGIN', 'LONGITUDE_ORIGIN']] = geolocate(i)[0], geolocate(i)[1]
    data.loc[data.CITY_DESTINATION == i, ['LATITUDE_DESTINATION', 'LONGITUDE_DESTINATION']] = geolocate(i)[0], geolocate(i)[1]

In [20]:
cities_origin_without_coords = data.loc[data.LATITUDE_ORIGIN.isna() | data.LONGITUDE_ORIGIN.isna(),
                                        'CITY_ORIGIN'].unique()
cities_destination_without_coords = data.loc[data.LATITUDE_DESTINATION.isna() | data.LONGITUDE_DESTINATION.isna(),
                                             'CITY_DESTINATION'].unique()

print('Origin cities without coordinates:', cities_origin_without_coords)
print('Destination cities without coordinates:', cities_destination_without_coords)

Origin cities without coordinates: []
Destination cities without coordinates: []


All our flights have coordinates so let's now create time zones for those flights.

In [21]:
tf = TimezoneFinder().timezone_at
data['TIMEZONE_DESTINATION'] = data[(data.LONGITUDE_DESTINATION.isna()== False) & (data.LATITUDE_DESTINATION.isna()== False)]\
    .apply(lambda x: tf(lng=x['LONGITUDE_DESTINATION'], lat=x['LATITUDE_DESTINATION']),axis=1)
data['TIMEZONE_ORIGIN'] = data[(data.LONGITUDE_ORIGIN.isna()== False) & (data.LATITUDE_ORIGIN.isna()== False)]\
    .apply(lambda x: tf(lng=x['LONGITUDE_ORIGIN'], lat=x['LATITUDE_ORIGIN']),axis=1)

In [22]:
print('Missing values of time zone origin:',data.TIMEZONE_ORIGIN.isna().sum())
print('Missing values of time zone destination:', data.TIMEZONE_DESTINATION.isna().sum())

Missing values of time zone origin: 153
Missing values of time zone destination: 154


It occurs that not all timezones has been assigned. Let's check which cities and states have missing time zones.

In [23]:
print('Cities of origin with missing values:', data.loc[data.TIMEZONE_ORIGIN.isna(), 'CITY_ORIGIN'].unique())
print('Cities of destination with missing values:', data.loc[data.TIMEZONE_DESTINATION.isna(), 'CITY_DESTINATION'].unique())
print('States of origin with missing values:', data.loc[data.TIMEZONE_ORIGIN.isna(), 'STATE_ORIGIN'].unique())
print('States of destination with missing values:', data.loc[data.TIMEZONE_DESTINATION.isna(), 'STATE_DESTINATION'].unique())

Cities of origin with missing values: ['Agana' 'Pago Pago']
Cities of destination with missing values: ['Agana' 'Pago Pago']
States of origin with missing values: ['GU' 'AS']
States of destination with missing values: ['GU' 'AS']


Let's determine now the time zones of flights with missing values based on their states. We can examine that with us package.

In [24]:
states_missing_tz = data.loc[data.TIMEZONE_DESTINATION.isna(), 'STATE_DESTINATION'].unique()
for i in range(len(states_missing_tz)):
    print('Time zones in', states_missing_tz[i], ':', us.states.lookup(states_missing_tz[i]).time_zones)

Time zones in GU : ['Pacific/Guam']
Time zones in AS : ['Pacific/Samoa']


States with missing time zones have only a single time zone. Let's encode time zones for those states.

In [25]:
states_tz = []
for i in range(len(states_missing_tz)):
    states_tz.append(us.states.lookup(states_missing_tz[i]).time_zones[0])

timezones = pd.DataFrame({'STATE': states_missing_tz, 'TIMEZONE': states_tz})

for state in timezones.STATE:
    for timezone in timezones.TIMEZONE:
        data.loc[data.STATE_DESTINATION == state, 'TIMEZONE_DESTINATION'] = timezone
        data.loc[data.STATE_ORIGIN == state, 'TIMEZONE_ORIGIN'] = timezone

In [26]:
data.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,STATE_ORIGIN,LATITUDE_ORIGIN,LONGITUDE_ORIGIN,CITY_ORIGIN,STATE_DESTINATION,LATITUDE_DESTINATION,LONGITUDE_DESTINATION,CITY_DESTINATION,SCHEDULED_DEPARTURE_HH,SCHEDULED_DEPARTURE_MM,DEPARTURE_TIME_HH,DEPARTURE_TIME_MM,SCHEDULED_ARRIVAL_HH,SCHEDULED_ARRIVAL_MM,WHEELS_OFF_HH,WHEELS_OFF_MM,ARRIVAL_TIME_HH,ARRIVAL_TIME_MM,ARRIVAL_TIME_ORIGIN_TZ,TIMEZONE_DESTINATION,TIMEZONE_ORIGIN
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,2015-01-01 00:05:00,2014-12-31 23:54:00,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,NaN,NaN,NaN,NaN,NaN,AK,61.17432,-149.99619,Anchorage,WA,47.44898,-122.30931,Seattle,0,5,23,54,4,30,0,15,4,8,2015-01-01 03:08:00,America/Los_Angeles,America/Anchorage
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,2015-01-01 00:10:00,2015-01-01 00:02:00,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,NaN,NaN,NaN,NaN,NaN,CA,33.94254,-118.40807,Los Angeles,FL,26.68316,-80.09559,West Palm Beach,0,10,0,2,7,50,0,14,7,41,2015-01-01 04:41:00,America/New_York,America/Los_Angeles
2,2015,1,1,4,US,840,N171US,SFO,CLT,2015-01-01 00:20:00,2015-01-01 00:18:00,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,NaN,NaN,NaN,NaN,NaN,CA,37.61900,-122.37484,San Francisco,NC,35.21401,-80.94313,Charlotte,0,20,0,18,8,6,0,34,8,11,2015-01-01 05:11:00,America/New_York,America/Los_Angeles
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,2015-01-01 00:20:00,2015-01-01 00:15:00,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,NaN,NaN,NaN,NaN,NaN,CA,33.94254,-118.40807,Los Angeles,FL,25.79325,-80.29056,Miami,0,20,0,15,8,5,0,30,7,56,2015-01-01 04:56:00,America/New_York,America/Los_Angeles
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,2015-01-01 00:25:00,2015-01-01 00:24:00,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,NaN,NaN,NaN,NaN,NaN,WA,47.44898,-122.30931,Seattle,AK,61.17432,-149.99619,Anchorage,0,25,0,24,3,20,0,35,2,59,2015-01-01 03:59:00,America/Anchorage,America/Los_Angeles


# NORMALIZED TIMES OF ARRIVAL AND DEPARTURE


We can derive arrival time in the destination time zone by using information from:
* ARRIVAL_TIME - we will take hours and minutes from this variable
* ARRIVAL_TIME_ORIGIN_TZ - we will take day and month values from it.

This approach however doesn't include the case where the day of an event is different than in the origin time zone.

Based on the knowledge that time zones in US differ by 6 hours at most, we can derive correct arrival times by following transformations:
* if the difference between ARRIVAL_TIME_ORIGIN_TZ and ARRIVAL_TIME hours is greater than 6 then we add a day (to avoid a situation in which origin airport's arrival time is from previous day and real arrival time is from the next day)
* if the difference between ARRIVAL_TIME_ORIGIN_TZ and ARRIVAL_TIME hours is less than -6 then we subtract a day (a situation where origin airport's arrival time is from the next day).

In [27]:
def correct_arrival(data):
    data.drop('ARRIVAL_TIME', axis = 1, inplace = True)
    arrival_day_to_small = data.loc[(data.ARRIVAL_TIME_ORIGIN_TZ.dt.hour - data.ARRIVAL_TIME_HH) > 6,
                                    ['ARRIVAL_TIME_ORIGIN_TZ', 'ARRIVAL_TIME_HH']]
    arrival_day_to_big = data.loc[(data.ARRIVAL_TIME_ORIGIN_TZ.dt.hour - data.ARRIVAL_TIME_HH) < -6,
                                    ['ARRIVAL_TIME_ORIGIN_TZ', 'ARRIVAL_TIME_HH']]
    arrival_day_correct = data[~data.index.isin(arrival_day_to_big.index | arrival_day_to_small.index)]

    data.loc[data.index.isin(arrival_day_to_small.index), 'ARRIVAL_TIME'] =\
               pd.to_datetime({'year': arrival_day_to_small.ARRIVAL_TIME_ORIGIN_TZ.dt.year,
              'month': arrival_day_to_small.ARRIVAL_TIME_ORIGIN_TZ.dt.month,
              'day': arrival_day_to_small.ARRIVAL_TIME_ORIGIN_TZ.dt.day,
              'hour': arrival_day_to_small.ARRIVAL_TIME_HH,
              'minute': arrival_day_to_small.ARRIVAL_TIME_ORIGIN_TZ.dt.minute
               }) + pd.Timedelta(days = 1)

    data.loc[data.index.isin(arrival_day_to_big.index), 'ARRIVAL_TIME'] =\
               pd.to_datetime({'year': arrival_day_to_big.ARRIVAL_TIME_ORIGIN_TZ.dt.year,
              'month': arrival_day_to_big.ARRIVAL_TIME_ORIGIN_TZ.dt.month,
              'day': arrival_day_to_big.ARRIVAL_TIME_ORIGIN_TZ.dt.day,
              'hour': arrival_day_to_big.ARRIVAL_TIME_HH,
              'minute': arrival_day_to_big.ARRIVAL_TIME_ORIGIN_TZ.dt.minute
               }) - pd.Timedelta(days = 1)

    data.loc[data.index.isin(arrival_day_correct.index), 'ARRIVAL_TIME'] =\
               pd.to_datetime({'year': arrival_day_correct.ARRIVAL_TIME_ORIGIN_TZ.dt.year,
              'month': arrival_day_correct.ARRIVAL_TIME_ORIGIN_TZ.dt.month,
              'day': arrival_day_correct.ARRIVAL_TIME_ORIGIN_TZ.dt.day,
              'hour': arrival_day_correct.ARRIVAL_TIME_HH,
              'minute': arrival_day_correct.ARRIVAL_TIME_ORIGIN_TZ.dt.minute
               })

In [28]:
correct_arrival(data)

Before creating normalized departure and arrival times let's check if all the dates are correct. In majority of US states there exists daylight saving time. According to wikipedia.org:

* "Most areas of the United States and Canada observe daylight saving time (DST), the exceptions being Arizona (except for the Navajo, who do observe daylight saving time on tribal lands), Hawaii, and the overseas territories of American Samoa, Guam, the Northern Mariana Islands, Puerto Rico, and the United States Virgin Islands. The Uniform Time Act of 1966 established the system of uniform daylight saving time throughout the US."

* "In the U.S., daylight saving time starts on the second Sunday in March and ends on the first Sunday in November, with the time changes taking place at 2:00 a.m. local time. With a mnemonic word play referring to seasons, clocks "spring forward, fall back"—that is, in springtime the clocks are moved forward from 2:00 a.m. to 3:00 a.m. and in fall they are moved back from 2:00 a.m. to 1:00 a.m."

In 2015 DST started on the 8th of March and ended on the 1st of November so there shouldn't be any observations between 2:00 a.m. and 2:59 a.m on 8th of March for states where DST exists.

https://en.wikipedia.org/wiki/Daylight_saving_time_in_the_United_States

In [29]:
timezones_without_DST = ['America/Phoenix', 'America/Puerto_Rico', 'Pacific/Guam', 'Pacific/Samoa', 'Pacific/Honolulu']


In [30]:
data.loc[~(data.TIMEZONE_DESTINATION.isin(timezones_without_DST)) &
        (data.ARRIVAL_TIME.between('2015-03-08 02:00:00', '2015-03-08 03:00:00')),
        ['CITY_DESTINATION', 'TIMEZONE_ORIGIN', 'TIMEZONE_DESTINATION', 'DEPARTURE_TIME', 'ARRIVAL_TIME']]

,CITY_DESTINATION,TIMEZONE_ORIGIN,TIMEZONE_DESTINATION,DEPARTURE_TIME,ARRIVAL_TIME
960287,Atlanta,America/Los_Angeles,America/New_York,2015-03-07 18:59:00,2015-03-08 02:15:00
964903,Miami,America/Chicago,America/New_York,2015-03-07 23:11:00,2015-03-08 02:41:00
964985,Atlanta,America/New_York,America/New_York,2015-03-07 23:15:00,2015-03-08 02:02:00
967398,Seattle,America/Anchorage,America/Los_Angeles,2015-03-07 21:32:00,2015-03-08 02:41:00
967675,New York,America/New_York,America/New_York,2015-03-08 00:05:00,2015-03-08 02:34:00
967787,Fairbanks,America/Los_Angeles,America/Anchorage,2015-03-07 23:08:00,2015-03-08 02:49:00


There are some wrong values for arrival time. Let's just switch those times one hour ahead.

In [31]:
data.loc[~(data.TIMEZONE_DESTINATION.isin(timezones_without_DST)) &
        (data.ARRIVAL_TIME.between('2015-03-08 02:00:00', '2015-03-08 03:00:00')),
        ['ARRIVAL_TIME']] =\
data.loc[~(data.TIMEZONE_DESTINATION.isin(timezones_without_DST)) &
        (data.ARRIVAL_TIME.between('2015-03-08 02:00:00', '2015-03-08 03:00:00')),
        ['ARRIVAL_TIME']] + pd.Timedelta(hours = 1)

In [32]:
data.loc[~(data.TIMEZONE_ORIGIN.isin(timezones_without_DST)) &
        (data.DEPARTURE_TIME.between('2015-03-08 02:00:00', '2015-03-08 03:00:00')),
        ['CITY_DESTINATION', 'TIMEZONE_DESTINATION', 'DEPARTURE_TIME', 'ELAPSED_TIME', 'ARRIVAL_TIME']]

,CITY_DESTINATION,TIMEZONE_DESTINATION,DEPARTURE_TIME,ELAPSED_TIME,ARRIVAL_TIME


There aren't any wrong times in departures anymore. We can finally create normalized times. All dates will be transformed to UTC±00:00.

In [33]:
data['DEPARTURE_TIME_NORMALIZED'] = data.groupby('TIMEZONE_ORIGIN')['DEPARTURE_TIME']\
    .transform(lambda x: x.dt.tz_localize(x.name, ambiguous = True).dt.tz_convert('UTC').dt.tz_localize(None))

data['ARRIVAL_TIME_NORMALIZED'] = data.groupby('TIMEZONE_DESTINATION')['ARRIVAL_TIME']\
    .transform(lambda x: x.dt.tz_localize(x.name, ambiguous = True).dt.tz_convert('UTC').dt.tz_localize(None))

Normalized times has been created.

We have still one more problem remaining. Hour-based columns have at midnight values for hour assigned as 24, which is the same as 0. Let's assign 0 values for such cases in order to avoid having 2 different values for the same hour.

In [34]:
for col in data.columns[data.columns.str.endswith('HH')]:
    data.loc[data[col] == 24, col] = 0

# CREATING AND DROPPING VARIABLES

Before moving to a visualization part we can also create a few new variables.
Let's create routes based on origin and destination airports and routes by airports states.

In [35]:
data['ROUTE'] = data.ORIGIN_AIRPORT + '_' + data.DESTINATION_AIRPORT
data['ROUTE_STATES'] = data.STATE_ORIGIN + '_' + data.STATE_DESTINATION

Lastly, we can remove variables that are no longer useful:
* minutes of dates aren't much informative
* time zones are no longer needed. We can obtain similar, but more precise information from airports and states
* cities are nearly the same as airports with only a few airports that are in the same city
* arrival time in origin time zone doesn't keep any useful information
* year (there is only 2015)

In [36]:
data.drop(list(data.columns[data.columns.str.endswith('MM')]) +
          ['TIMEZONE_ORIGIN', 'TIMEZONE_DESTINATION', 'CITY_ORIGIN', 'CITY_DESTINATION', 'ARRIVAL_TIME_ORIGIN_TZ', 'YEAR'],
          axis = 1, inplace = True)

In [37]:
data.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,STATE_ORIGIN,LATITUDE_ORIGIN,LONGITUDE_ORIGIN,STATE_DESTINATION,LATITUDE_DESTINATION,LONGITUDE_DESTINATION,SCHEDULED_DEPARTURE_HH,DEPARTURE_TIME_HH,SCHEDULED_ARRIVAL_HH,WHEELS_OFF_HH,ARRIVAL_TIME_HH,ARRIVAL_TIME,DEPARTURE_TIME_NORMALIZED,ARRIVAL_TIME_NORMALIZED,ROUTE,ROUTE_STATES
0,1,1,4,AS,98,N407AS,ANC,SEA,2015-01-01 00:05:00,2014-12-31 23:54:00,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,-22.0,NaN,NaN,NaN,NaN,NaN,AK,61.17432,-149.99619,WA,47.44898,-122.30931,0,23,4,0,4,2015-01-01 04:08:00,2015-01-01 08:54:00,2015-01-01 12:08:00,ANC_SEA,AK_WA
1,1,1,4,AA,2336,N3KUAA,LAX,PBI,2015-01-01 00:10:00,2015-01-01 00:02:00,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,-9.0,NaN,NaN,NaN,NaN,NaN,CA,33.94254,-118.40807,FL,26.68316,-80.09559,0,0,7,0,7,2015-01-01 07:41:00,2015-01-01 08:02:00,2015-01-01 12:41:00,LAX_PBI,CA_FL
2,1,1,4,US,840,N171US,SFO,CLT,2015-01-01 00:20:00,2015-01-01 00:18:00,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,5.0,NaN,NaN,NaN,NaN,NaN,CA,37.61900,-122.37484,NC,35.21401,-80.94313,0,0,8,0,8,2015-01-01 08:11:00,2015-01-01 08:18:00,2015-01-01 13:11:00,SFO_CLT,CA_NC
3,1,1,4,AA,258,N3HYAA,LAX,MIA,2015-01-01 00:20:00,2015-01-01 00:15:00,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,-9.0,NaN,NaN,NaN,NaN,NaN,CA,33.94254,-118.40807,FL,25.79325,-80.29056,0,0,8,0,7,2015-01-01 07:56:00,2015-01-01 08:15:00,2015-01-01 12:56:00,LAX_MIA,CA_FL
4,1,1,4,AS,135,N527AS,SEA,ANC,2015-01-01 00:25:00,2015-01-01 00:24:00,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,-21.0,NaN,NaN,NaN,NaN,NaN,WA,47.44898,-122.30931,AK,61.17432,-149.99619,0,0,3,0,2,2015-01-01 02:59:00,2015-01-01 08:24:00,2015-01-01 11:59:00,SEA_ANC,WA_AK


In [38]:
data.to_pickle('data.pkl')